# Lecture Notes Toolkit - Finite Markov Chains (Chapter 7)

## 📚 What are Markov Chains?

A **Markov chain** is a mathematical model that describes a sequence of events where the probability of each event depends **only on the state of the previous event**, not on the entire history. This is called the **Markov property** or "memorylessness."

### 🎯 Real-World Examples:
- **Weather**: Tomorrow's weather depends mainly on today's weather, not last week's
- **Stock prices**: Tomorrow's price movements depend on today's price
- **Web browsing**: The next page you visit depends on your current page (basis of Google's PageRank!)
- **Text generation**: The next word depends on the current word (simple language models)
- **Games**: Moving between game states based on current position

### 📖 What This Notebook Covers

This notebook provides a complete toolkit for working with **finite Markov chains** (chains with a countable number of states). We cover both theory and practical implementations based on notes from pages 110-122.

---

## 🗂️ Content Overview

### **Part 1: Foundations** (Sections 0-1)
✅ **Core Concepts & Validation**
- What is a transition matrix `P`?
- Row-stochastic property (rows sum to 1)
- Helper functions for validation and normalization

✅ **Distributions Over Time: `μ_t`**
- Computing state distributions after `t` steps
- Homogeneous chains (constant `P`) vs. inhomogeneous chains (time-varying `P_t`)
- Expected values of functions at time `t`

---

### **Part 2: Simulation** (Section 2)
✅ **Random Mapping Representation (RMR)**
- How to simulate Markov chains efficiently
- Inversion sampling method (Theorem 7.14)
- Simulating both homogeneous and inhomogeneous chains

---

### **Part 3: Structure & Properties** (Sections 3-4)
✅ **Irreducibility & Communication**
- Can you reach any state from any other state?
- Finding communication classes
- Why irreducibility matters for long-run behavior

✅ **Return Times, Period & Aperiodicity**
- How often does a chain return to a state?
- Periodic vs. aperiodic chains
- Impact on convergence to equilibrium

---

### **Part 4: Long-Run Behavior** (Sections 5-6)
✅ **Stationary Distributions: `π`**
- The equilibrium distribution (`πP = π`)
- When does it exist? When is it unique?
- Two methods: solving linear systems vs. power iteration

✅ **Reversibility (Detailed Balance)**
- What does it mean for a chain to be reversible?
- Connection between reversibility and stationarity
- Physical interpretation

---

### **Part 5: Applications** (Sections 7-9)
✅ **Random Walks on Graphs**
- Modeling movement on networks
- Computing stationary distributions from graph structure
- PageRank-style random surfer with teleportation

✅ **Exercise 1: Text as a Markov Chain**
- Building word-level language models
- Estimating transition probabilities from text
- Generating sentences

✅ **Exercise 2: Branching Processes**
- Population dynamics and extinction
- Galton-Watson processes
- Estimating extinction times

---

## 🎓 Learning Approach

This notebook is designed for **step-by-step learning**:
1. **Read the explanations** in markdown cells carefully
2. **Study the code** with inline comments explaining each step
3. **Run the examples** to see concepts in action
4. **Modify parameters** in demo cells to build intuition
5. **Reuse functions** for your own problems (everything is modular!)

Let's begin! 🚀

In [ ]:
# Core imports
import numpy as np
import math
from dataclasses import dataclass
from typing import Callable, Dict, Iterable, List, Optional, Sequence, Tuple

import matplotlib.pyplot as plt


## 0) Core Helpers: Understanding Transition Matrices

### 🎯 What is a Transition Matrix?

A **transition matrix** `P` encodes the probabilities of moving between states in a Markov chain.

**Structure:**
- `P` is a square matrix of size `k × k` (where `k` = number of states)
- Entry `P[i,j]` = probability of moving from state `i` to state `j` in one step
- States are typically numbered `0, 1, 2, ..., k-1`

**Example:** 3-state weather model (Sunny=0, Cloudy=1, Rainy=2)
```
       To: S    C    R
From S:  [0.7  0.2  0.1]   ← If sunny today, 70% chance sunny tomorrow
From C:  [0.3  0.4  0.3]   ← If cloudy today, 40% chance cloudy tomorrow  
From R:  [0.2  0.3  0.5]   ← If rainy today, 50% chance rainy tomorrow
```

---

### 📏 The Row-Stochastic Property

For `P` to be a valid transition matrix, it must be **row-stochastic**:

1. **Non-negative entries:** `P[i,j] ≥ 0` (probabilities can't be negative!)
2. **Rows sum to 1:** `Σⱼ P[i,j] = 1` (from any state, total probability must be 100%)

This makes sense: from state `i`, you must go *somewhere* with total probability 1.

---

### ⚠️ Why Validate?

Before using a matrix as `P`, we must ensure it's row-stochastic:
- **Normalization errors:** Floating-point arithmetic can cause small deviations
- **User input errors:** Manually entered matrices might have mistakes
- **Construction bugs:** Programmatically built matrices might have logical errors

The functions below handle validation, normalization, and error-checking automatically.

In [ ]:
def normalize_rows(P: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    """
    Normalize each row of matrix P to sum to 1.
    
    This ensures the matrix is row-stochastic (each row is a probability distribution).
    
    Args:
        P: Input matrix (will be converted to float array)
        eps: Small threshold to avoid division by zero (default 1e-12)
        
    Returns:
        Normalized matrix where each row sums to 1
        
    How it works:
        1. Compute the sum of each row
        2. If a row sum is < eps (essentially zero), replace with 1.0 to avoid division by zero
        3. Divide each row by its sum to normalize
        
    Example:
        >>> P = np.array([[1, 2], [3, 1]])
        >>> normalize_rows(P)
        array([[0.333, 0.667],   # row 0: sum was 3, now 1/3 and 2/3
               [0.75,  0.25]])    # row 1: sum was 4, now 3/4 and 1/4
    """
    P = np.asarray(P, dtype=float)  # Convert to numpy float array
    row_sums = P.sum(axis=1, keepdims=True)  # Sum each row, keep 2D shape for broadcasting
    row_sums = np.where(row_sums < eps, 1.0, row_sums)  # Replace near-zero sums with 1.0
    return P / row_sums  # Divide each element by its row sum

def is_row_stochastic(P: np.ndarray, tol: float = 1e-9) -> bool:
    """
    Check if matrix P is row-stochastic (valid transition matrix).
    
    A matrix is row-stochastic if:
        - All entries are non-negative (>= 0)
        - Each row sums to 1 (within tolerance)
    
    Args:
        P: Matrix to check
        tol: Tolerance for row sums being equal to 1 (default 1e-9)
        
    Returns:
        True if P is row-stochastic, False otherwise
        
    Example:
        >>> P = np.array([[0.7, 0.3], [0.4, 0.6]])
        >>> is_row_stochastic(P)
        True
        >>> P_bad = np.array([[0.5, 0.3], [0.4, 0.6]])  # first row sums to 0.8
        >>> is_row_stochastic(P_bad)
        False
    """
    P = np.asarray(P, dtype=float)
    
    # Check 1: No negative entries
    if np.any(P < -tol):
        return False
    
    # Check 2: Each row sums to 1 (within tolerance)
    row_sums = P.sum(axis=1)
    return bool(np.all(np.abs(row_sums - 1.0) <= tol))

def ensure_row_stochastic(P: np.ndarray, tol: float = 1e-9) -> np.ndarray:
    """
    Ensure matrix P is row-stochastic by normalizing if needed.
    
    This is the main validation function used throughout the toolkit.
    It normalizes rows to sum to 1 and raises an error if there are negative entries.
    
    Args:
        P: Input matrix (potential transition matrix)
        tol: Tolerance for validation checks
        
    Returns:
        Normalized row-stochastic matrix
        
    Raises:
        ValueError: If P has negative entries or cannot be normalized to valid probabilities
        
    Usage:
        Call this at the start of every function that requires a transition matrix.
        It ensures safety and consistency across all operations.
    """
    P = np.asarray(P, dtype=float)
    
    # Check for negative entries (probabilities can't be negative!)
    if np.any(P < -tol):
        raise ValueError("Transition matrix has negative entries.")
    
    # Normalize rows to sum to 1
    Pn = normalize_rows(P)
    
    # Verify the result is actually row-stochastic
    if not is_row_stochastic(Pn, tol=1e-6):
        raise ValueError("Could not normalize rows to a valid transition matrix.")
    
    return Pn

## 1) Distributions Over Time: Understanding `μ_t`

### 🎯 What is `μ_t`?

`μ_t` (pronounced "mu at time t") is a **probability distribution over states at time `t`**.

- `μ_t[i]` = probability of being in state `i` at time `t`
- `μ_t` is a row vector with `k` elements (one per state)
- `μ_t` sums to 1 (it's a probability distribution)
- `μ_0` is the **initial distribution** (where we start)

**Example:**
```
μ_0 = [1.0, 0.0, 0.0]  ← Start in state 0 with certainty
μ_0 = [0.5, 0.3, 0.2]  ← Start in state 0 with 50%, state 1 with 30%, state 2 with 20%
```

---

### 📐 The Chapman-Kolmogorov Equation

The fundamental relationship is:
$$\mu_{t+1} = \mu_t P$$

This tells us: **multiply the current distribution by the transition matrix to get the next distribution.**

**Intuition:** The probability of being in state `j` at time `t+1` is the sum over all states `i` of:
- (probability of being in state `i` at time `t`) × (probability of transitioning from `i` to `j`)

---

### 🔄 Homogeneous Chains: When `P` is Constant

If the transition matrix doesn't change over time, we can apply it repeatedly:
$$\mu_t = \mu_0 P^t$$

Where `P^t` means "`P` multiplied by itself `t` times" (matrix power).

**Why this works:**
- `μ_1 = μ_0 P`
- `μ_2 = μ_1 P = (μ_0 P) P = μ_0 P²`
- `μ_3 = μ_2 P = μ_0 P³`
- And so on...

**Physical interpretation:** `P^t[i,j]` = probability of going from state `i` to state `j` in exactly `t` steps.

---

### 🔀 Inhomogeneous Chains: When `P` Changes Over Time

Sometimes the transition probabilities change with time (e.g., seasonal weather patterns):
- At time 1, use transition matrix `P_1`
- At time 2, use transition matrix `P_2`
- etc.

Then:
$$\mu_t = \mu_0 P_1 P_2 \cdots P_t$$

We multiply the matrices in sequence (order matters!).

---

### 📊 Expected Values at Time `t`

If you have a function `f` that assigns a value to each state (e.g., temperature in a weather model), you can compute the expected value at time `t`:
$$\mathbb{E}[f(X_t)] = (P^t f)[x]$$

This is useful for:
- Average rewards in reinforcement learning
- Expected costs/benefits at future times
- Long-run averages

In [ ]:
def n_step_transition(P: np.ndarray, t: int) -> np.ndarray:
    """
    Compute the t-step transition matrix P^t.
    
    P^t[i,j] = probability of going from state i to state j in exactly t steps.
    
    Args:
        P: One-step transition matrix (k × k)
        t: Number of steps (must be >= 0)
        
    Returns:
        P^t: The t-step transition matrix
        
    Special cases:
        - t=0: Returns identity matrix (stay in same state)
        - t=1: Returns P itself
        - t=2: Returns P @ P (two-step transitions)
        
    Example:
        If P = [[0.5, 0.5], [0.3, 0.7]], then P^2 tells us two-step probabilities.
    """
    if t < 0:
        raise ValueError("t must be >= 0.")
    P = ensure_row_stochastic(P)
    return np.linalg.matrix_power(P, t)  # Efficiently compute P^t

def dist_after_t_homogeneous(mu0: np.ndarray, P: np.ndarray, t: int) -> np.ndarray:
    """
    Compute the distribution μ_t = μ_0 P^t for a homogeneous Markov chain.
    
    This answers: "If I start with distribution μ_0, what distribution will I have after t steps?"
    
    Args:
        mu0: Initial distribution (length k, sums to 1)
        P: Transition matrix (k × k, constant over time)
        t: Number of steps
        
    Returns:
        μ_t: Distribution at time t (length k, sums to 1)
        
    Example:
        >>> mu0 = np.array([1.0, 0.0])  # Start in state 0
        >>> P = np.array([[0.7, 0.3], [0.4, 0.6]])
        >>> dist_after_t_homogeneous(mu0, P, t=1)
        array([0.7, 0.3])  # After 1 step, 70% in state 0, 30% in state 1
    """
    if t < 0:
        raise ValueError("t must be >= 0.")
    P = ensure_row_stochastic(P)
    mu0 = np.asarray(mu0, dtype=float).reshape(1, -1)  # Ensure row vector shape
    
    # Validate dimensions match
    if mu0.shape[1] != P.shape[0]:
        raise ValueError("mu0 length must match P dimension.")
    
    # Compute P^t then multiply: μ_0 @ P^t
    Pt = n_step_transition(P, t)
    return (mu0 @ Pt).ravel()  # Return as 1D array

def dist_after_t_inhomogeneous(mu0: np.ndarray, Ps: Sequence[np.ndarray], t: int) -> np.ndarray:
    """
    Compute μ_t = μ_0 P_1 P_2 ... P_t for an inhomogeneous chain.
    
    Use this when transition probabilities change over time (e.g., seasonal effects).
    
    Args:
        mu0: Initial distribution
        Ps: Sequence of transition matrices [P_1, P_2, ..., P_T]
            Ps[0] is the transition matrix used at step 1
            Ps[1] is used at step 2, etc.
        t: Number of steps to simulate
        
    Returns:
        μ_t: Distribution after t steps
        
    Note:
        We need at least t matrices in Ps to compute t steps.
    """
    if t < 0:
        raise ValueError("t must be >= 0.")
    mu0 = np.asarray(mu0, dtype=float).reshape(1, -1)
    
    # Special case: at time 0, return initial distribution
    if t == 0:
        return mu0.ravel()
    
    # Check we have enough transition matrices
    if t > len(Ps):
        raise ValueError("Need at least t matrices in Ps.")
    
    k = mu0.shape[1]
    mu = mu0
    
    # Multiply through the sequence: μ_0 @ P_1 @ P_2 @ ... @ P_t
    for i in range(t):
        Pi = ensure_row_stochastic(Ps[i])
        if Pi.shape != (k, k):
            raise ValueError("All matrices must match state dimension.")
        mu = mu @ Pi  # Update distribution by multiplying with next P
    
    return mu.ravel()

def expected_f_at_t(P: np.ndarray, f: np.ndarray, t: int) -> np.ndarray:
    """
    Compute expected value of function f at time t, starting from each state.
    
    Returns g where g[x] = E_x[f(X_t)] = expected value of f(X_t) starting from state x.
    This is computed as (P^t @ f)[x].
    
    Args:
        P: Transition matrix
        f: Function values at each state (length k)
        t: Time step
        
    Returns:
        Array g where g[x] = expected value of f at time t starting from state x
        
    Example:
        If states are temperatures and f = [10, 20, 30] (degrees),
        then g[0] tells us the expected temperature at time t if we start in state 0.
    """
    P = ensure_row_stochastic(P)
    f = np.asarray(f, dtype=float).reshape(-1)
    
    if f.shape[0] != P.shape[0]:
        raise ValueError("f length must match P dimension.")
    
    # P^t gives t-step transition probabilities
    # (P^t @ f)[x] = sum over all states y of: P^t[x,y] * f[y]
    # This is the expected value of f, starting from x, after t steps
    Pt = n_step_transition(P, t)
    return Pt @ f

### Mini demo (replace `P_demo` / `mu0`)

We create a 3-state chain and show `μ_t`.


In [ ]:
P_demo = np.array([
    [0.1, 0.6, 0.3],
    [0.2, 0.7, 0.1],
    [0.5, 0.2, 0.3],
], dtype=float)

mu0 = np.array([1.0, 0.0, 0.0])  # start surely in state 0

for t in [0, 1, 2, 5, 10]:
    print(t, dist_after_t_homogeneous(mu0, P_demo, t))


## 2) Simulation via Random Mapping Representation (RMR)

### 🎯 The Goal: Simulate Random Paths

We want to generate actual sample paths (trajectories) of a Markov chain:
```
X_0 → X_1 → X_2 → X_3 → ...
```
where each transition follows the probabilities in `P`.

**Why simulate?**
- Estimate long-run behavior empirically
- Generate sample data for testing
- Visualize typical trajectories
- Monte Carlo estimation of expectations

---

### 🎲 The Challenge: How to Sample Transitions?

Given we're in state `i`, we need to randomly choose the next state `j` with probabilities `P[i,j]`.

**Example:** If `P[i,:] = [0.1, 0.6, 0.3]`, we need:
- 10% chance of choosing state 0
- 60% chance of choosing state 1  
- 30% chance of choosing state 2

---

### 💡 Solution: Inversion Sampling (Theorem 7.14)

The **Random Mapping Representation (RMR)** uses **inversion sampling** to convert uniform random numbers into state transitions.

**The Algorithm:**

1. **Precompute cumulative probabilities** for each row of `P`:
   ```
   CDF[i,0] = P[i,0]
   CDF[i,1] = P[i,0] + P[i,1]
   CDF[i,2] = P[i,0] + P[i,1] + P[i,2]
   ...
   CDF[i,k-1] = 1.0
   ```

2. **Draw a uniform random number** `u ~ Uniform(0,1)`

3. **Find the first index** `j` where `u ≤ CDF[i,j]`

4. **Next state is `j`**

---

### 🔍 Why This Works: Visual Intuition

Think of the interval [0,1] divided into segments proportional to probabilities:

```
Current state i has P[i,:] = [0.1, 0.6, 0.3]

[0.0 -------- 0.1|------- 0.7|---- 1.0]
     State 0       State 1    State 2
     (10%)          (60%)      (30%)
```

When we draw `u ~ Uniform(0,1)`:
- If `u ∈ [0.0, 0.1)` → choose state 0 (happens 10% of the time)
- If `u ∈ [0.1, 0.7)` → choose state 1 (happens 60% of the time)
- If `u ∈ [0.7, 1.0]` → choose state 2 (happens 30% of the time)

This exactly matches the probabilities in `P[i,:]`!

---

### ⚡ Efficiency

By precomputing the CDF for all rows, we can:
1. Generate many transitions quickly
2. Reuse the same RMR for multiple simulations
3. Use efficient binary search (`np.searchsorted`) to find the right state

This is **much faster** than alternatives like `np.random.choice` for large numbers of steps.

In [ ]:
@dataclass
class RMR:
    """
    Random Mapping Representation: Precomputed data for efficient simulation.
    
    Attributes:
        P: The transition matrix (k × k)
        cdf_rows: Cumulative distribution function for each row (k × k)
                  cdf_rows[i,j] = sum of P[i,0] + P[i,1] + ... + P[i,j]
    """
    P: np.ndarray
    cdf_rows: np.ndarray  # cumulative sums per row, shape (k,k)

def build_rmr(P: np.ndarray) -> RMR:
    """
    Build a Random Mapping Representation from a transition matrix.
    
    This precomputes the cumulative distribution functions (CDFs) for all rows,
    enabling fast sampling of next states.
    
    Args:
        P: Transition matrix (will be validated and normalized)
        
    Returns:
        RMR object containing P and precomputed CDFs
        
    Example:
        >>> P = np.array([[0.1, 0.6, 0.3], [0.5, 0.3, 0.2]])
        >>> rmr = build_rmr(P)
        >>> rmr.cdf_rows
        array([[0.1, 0.7, 1.0],     # cumulative: 0.1, 0.1+0.6=0.7, 0.7+0.3=1.0
               [0.5, 0.8, 1.0]])     # cumulative: 0.5, 0.5+0.3=0.8, 0.8+0.2=1.0
    """
    P = ensure_row_stochastic(P)  # Validate and normalize
    cdf = np.cumsum(P, axis=1)    # Cumulative sum along each row
    cdf[:, -1] = 1.0              # Fix last column to exactly 1.0 (avoid floating point drift)
    return RMR(P=P, cdf_rows=cdf)

def rmr_step(rmr: RMR, x: int, u: float) -> int:
    """
    Take one step in the Markov chain using the RMR and a uniform random number.
    
    Args:
        rmr: Random Mapping Representation (precomputed)
        x: Current state (0 ≤ x < k)
        u: Uniform random number in [0, 1]
        
    Returns:
        Next state j, chosen according to probabilities P[x,:]
        
    How it works:
        1. Get the CDF for row x: [CDF[x,0], CDF[x,1], ..., CDF[x,k-1]]
        2. Find the first index j where u ≤ CDF[x,j]
        3. This j is chosen with probability P[x,j]
        
    Example:
        If P[x,:] = [0.2, 0.5, 0.3], then CDF[x,:] = [0.2, 0.7, 1.0]
        - If u = 0.15 → j = 0 (since 0.15 ≤ 0.2)
        - If u = 0.5  → j = 1 (since 0.2 < 0.5 ≤ 0.7)
        - If u = 0.9  → j = 2 (since 0.7 < 0.9 ≤ 1.0)
    """
    row = rmr.cdf_rows[x]  # Get CDF for current state x
    # Binary search to find first j where u <= row[j]
    # searchsorted with side="right" finds the insertion point
    return int(np.searchsorted(row, u, side="right"))

def simulate_markov_chain(P: np.ndarray, x0: int, n_steps: int, seed: Optional[int] = 0) -> np.ndarray:
    """
    Simulate a trajectory of a homogeneous Markov chain.
    
    Generates a path: X_0, X_1, X_2, ..., X_{n_steps}
    
    Args:
        P: Transition matrix (k × k, constant over time)
        x0: Initial state (0 ≤ x0 < k)
        n_steps: Number of steps to simulate (length of output will be n_steps + 1)
        seed: Random seed for reproducibility (None for random behavior)
        
    Returns:
        Array of states [X_0, X_1, ..., X_{n_steps}] of length (n_steps + 1)
        
    Example:
        >>> P = np.array([[0.5, 0.5], [0.3, 0.7]])
        >>> path = simulate_markov_chain(P, x0=0, n_steps=5, seed=42)
        >>> path
        array([0, 1, 1, 0, 0, 1])  # Started in state 0, took 5 steps
    """
    if n_steps < 0:
        raise ValueError("n_steps must be >= 0.")
    
    P = ensure_row_stochastic(P)
    k = P.shape[0]
    
    if not (0 <= x0 < k):
        raise ValueError("x0 out of range.")
    
    # Build the RMR once (precompute CDFs)
    rmr = build_rmr(P)
    
    # Initialize random number generator
    rng = np.random.default_rng(seed)
    
    # Allocate array for the trajectory
    xs = np.empty(n_steps + 1, dtype=int)
    xs[0] = x0  # Set initial state
    
    x = x0
    # Generate each transition
    for t in range(1, n_steps + 1):
        u = rng.random()           # Draw uniform random number
        x = rmr_step(rmr, x, u)    # Use RMR to get next state
        xs[t] = x                   # Store it
    
    return xs

def simulate_inhomogeneous_chain(Ps: Sequence[np.ndarray], x0: int, seed: Optional[int] = 0) -> np.ndarray:
    """
    Simulate an inhomogeneous Markov chain (time-varying transition matrices).
    
    At each time step t, uses transition matrix Ps[t-1] to go from X_{t-1} to X_t.
    
    Args:
        Ps: Sequence of transition matrices [P_1, P_2, ..., P_T]
            Ps[0] is used for the first transition (X_0 → X_1)
            Ps[1] is used for the second transition (X_1 → X_2)
            etc.
        x0: Initial state
        seed: Random seed
        
    Returns:
        Array [X_0, X_1, ..., X_T] of length (T + 1), where T = len(Ps)
        
    Example:
        Use when transition probabilities change over time (e.g., seasonal patterns).
    """
    T = len(Ps)
    
    # Special case: no transitions
    if T == 0:
        return np.array([x0], dtype=int)
    
    k = Ps[0].shape[0]  # Number of states
    rng = np.random.default_rng(seed)
    
    # Allocate trajectory array
    xs = np.empty(T + 1, dtype=int)
    xs[0] = x0
    
    x = x0
    # At each time step, use the corresponding transition matrix
    for t in range(1, T + 1):
        Pt = ensure_row_stochastic(Ps[t-1])  # Get and validate P_t
        rmr = build_rmr(Pt)                   # Build RMR for this time step
        u = rng.random()                      # Draw random number
        x = rmr_step(rmr, x, u)               # Transition
        xs[t] = x
    
    return xs

In [ ]:
# Demo trajectory
traj = simulate_markov_chain(P_demo, x0=0, n_steps=25, seed=42)
traj


## 3) Irreducibility: Can We Reach Everywhere?

### 🎯 What is Irreducibility?

A Markov chain is **irreducible** if you can eventually reach any state from any other state (possibly in multiple steps).

**Formal definition:** State `i` **communicates** with state `j` (written `i ↔ j`) if:
- You can reach `j` from `i`: there exists some `t` where `P^t[i,j] > 0`
- You can reach `i` from `j`: there exists some `s` where `P^s[j,i] > 0`

The chain is **irreducible** if all states communicate with each other.

---

### 🗺️ Visual Examples

**Example 1: Irreducible chain (3 states)**
```
    ┌─→ 0 ←─┐
    │   ↓   │
    └── 1 → 2
```
From any state, you can eventually reach any other state. ✅ Irreducible

**Example 2: Reducible chain (2 separate groups)**
```
    0 ↔ 1       2 ↔ 3
    (Group A)   (Group B)
```
Once you're in Group A, you can never reach Group B (and vice versa). ❌ Not irreducible

**Example 3: Absorbing state**
```
    0 → 1 → 2
        ↓
        2 (self-loop, P[2,2]=1)
```
State 2 is **absorbing** (once you enter, you never leave). You can't return to states 0 or 1. ❌ Not irreducible

---

### 🔍 Communication Classes

If a chain is not irreducible, it splits into **communication classes**:
- **Class** = maximal set of states that all communicate with each other
- States in different classes don't communicate

**Example:**
```
States: {0, 1, 2, 3, 4}
P shows: 0↔1, 2↔3↔4
Classes: {0,1} and {2,3,4}
```

---

### 💡 Why Irreducibility Matters

1. **Unique stationary distribution:** Irreducible + aperiodic chains have exactly one stationary distribution `π`
2. **Convergence:** The distribution `μ_t` converges to `π` regardless of starting point
3. **Ergodic theorem:** Time averages equal space averages (law of large numbers for Markov chains)

If the chain is **not** irreducible:
- Multiple stationary distributions may exist
- Long-run behavior depends on where you start
- Different classes behave independently

---

### 🔧 How to Check: Graph Approach

View the chain as a directed graph:
- Vertices = states
- Edge `i → j` exists if `P[i,j] > 0`

The chain is irreducible ⟺ the graph is **strongly connected** (you can reach any vertex from any other vertex following directed edges).

In [ ]:
def adjacency_from_P(P: np.ndarray, tol: float = 0.0) -> List[List[int]]:
    P = np.asarray(P, dtype=float)
    k = P.shape[0]
    adj = [[] for _ in range(k)]
    for i in range(k):
        adj[i] = [j for j in range(k) if P[i, j] > tol]
    return adj

def _dfs(start: int, adj: List[List[int]]) -> List[bool]:
    seen = [False]*len(adj)
    stack = [start]
    while stack:
        v = stack.pop()
        if seen[v]:
            continue
        seen[v] = True
        for w in adj[v]:
            if not seen[w]:
                stack.append(w)
    return seen

def is_strongly_connected(adj: List[List[int]]) -> bool:
    n = len(adj)
    # check reachability from 0
    seen = _dfs(0, adj)
    if not all(seen):
        return False
    # build reverse graph
    radj = [[] for _ in range(n)]
    for i in range(n):
        for j in adj[i]:
            radj[j].append(i)
    seen_r = _dfs(0, radj)
    return all(seen_r)

def is_irreducible(P: np.ndarray, tol: float = 0.0) -> bool:
    P = ensure_row_stochastic(P)
    return is_strongly_connected(adjacency_from_P(P, tol=tol))

is_irreducible(P_demo)


## 4) Return Times, Period, and Aperiodicity

### 🎯 What is the Period of a State?

The **period** of a state `x` measures the "rhythm" of returns to that state.

**Definition:** 
- Let `T(x) = {t ≥ 1 : P^t[x,x] > 0}` be the set of all times when return is possible
- The **period** of `x` is `d(x) = gcd(T(x))` (greatest common divisor)

**Interpretation:**
- Period tells you the "cycle length" of returns
- If `d(x) = 1`, the state is **aperiodic** (returns can happen at irregular times)
- If `d(x) > 1`, the state is **periodic** (returns follow a fixed cycle)

---

### 📊 Examples

**Example 1: Aperiodic state**
```
State 0 can return to itself at times: {1, 2, 3, 4, 5, ...}
gcd(1, 2, 3, 4, 5, ...) = 1  ✅ Aperiodic
```
This happens when P[0,0] > 0 (self-loop exists).

**Example 2: Period 2 (deterministic cycle)**
```
     0 → 1
     ↑   ↓
     └───┘
```
- From state 0: must go to 1, then back to 0
- Can only return at times 2, 4, 6, 8, ...
- `gcd(2, 4, 6, 8, ...) = 2` ✅ Period = 2

**Example 3: Period 3**
```
     0 → 1 → 2
     ↑       ↓
     └───────┘
```
- From state 0: goes 0→1→2→0→1→2→...
- Returns possible at times 3, 6, 9, 12, ...
- `gcd(3, 6, 9, ...) = 3` ✅ Period = 3

**Example 4: Aperiodic (self-loop breaks periodicity)**
```
     0 → 1
     ↓ ↖ ↓
     └─→─┘
```
- State 0 can return in 2 steps (0→1→0) or 3 steps (0→0→0→0)
- `gcd(2, 3) = 1` ✅ Aperiodic

---

### 🔑 Key Facts

1. **Irreducible chains:** All states have the same period
   - If any one state is aperiodic, the entire chain is aperiodic

2. **Self-loops guarantee aperiodicity:** If P[x,x] > 0 for any state x, then d(x) = 1

3. **Bipartite graphs:** Chains on bipartite graphs (like checkerboards) have period 2

---

### 💡 Why Periodicity Matters

**Aperiodic chains (period = 1):**
- `μ_t` converges smoothly to stationary distribution `π`
- `P^t[i,j]` converges to `π[j]` as `t → ∞`
- Nice ergodic properties

**Periodic chains (period d > 1):**
- `μ_t` **oscillates** and never converges!
- Returns happen in cycles of length `d`
- Need to look at averaged behavior: `(P + P^2 + ... + P^d) / d`

**Example:** On a 2-state cycle (0 ↔ 1):
```
Start at state 0:
t=0: in state 0
t=1: in state 1  
t=2: in state 0
t=3: in state 1
... (keeps alternating, never settles down!)
```

---

### 🔧 Computational Approach

To find the period of state `x`:
1. Compute `P^1, P^2, P^3, ...` up to some horizon
2. Collect all `t` where `P^t[x,x] > 0`
3. Take GCD of these times

For irreducible chains, checking one state is enough (all states share the same period).

In [ ]:
def state_period(P: np.ndarray, x: int, max_t: int = 500, tol: float = 1e-15) -> Optional[int]:
    """Estimate period of state x by collecting return times up to max_t."""
    P = ensure_row_stochastic(P)
    k = P.shape[0]
    if not (0 <= x < k):
        raise ValueError("x out of range.")
    g = 0
    Pt = np.eye(k)
    for t in range(1, max_t + 1):
        Pt = Pt @ P
        if Pt[x, x] > tol:
            g = math.gcd(g, t) if g != 0 else t
    return g if g != 0 else None  # None means no return detected within max_t

def chain_period(P: np.ndarray, max_t: int = 500) -> Optional[int]:
    """If irreducible, all states share a period; return estimated common period."""
    P = ensure_row_stochastic(P)
    k = P.shape[0]
    periods = []
    for x in range(k):
        px = state_period(P, x, max_t=max_t)
        if px is not None:
            periods.append(px)
    if not periods:
        return None
    # if irreducible, these should match; we return gcd as a safe summary
    g = periods[0]
    for p in periods[1:]:
        g = math.gcd(g, p)
    return g

def is_aperiodic(P: np.ndarray, max_t: int = 500) -> bool:
    p = chain_period(P, max_t=max_t)
    return p == 1

chain_period(P_demo), is_aperiodic(P_demo)


## 5) Stationary Distribution: The Long-Run Equilibrium

### 🎯 What is a Stationary Distribution?

A **stationary distribution** `π` is a probability distribution over states that doesn't change under the Markov chain transition.

**Mathematical definition:**
- `π` is a row vector with `k` elements (one per state)
- `π[i] ≥ 0` for all `i`, and `Σᵢ π[i] = 1` (it's a probability distribution)
- **Stationarity property:** `πP = π`

**Intuition:** If the chain is in distribution `π` at time `t`, it will still be in distribution `π` at time `t+1`, `t+2`, etc. The distribution is "stuck" or "balanced."

---

### 🌊 Physical Interpretation: Flow Equilibrium

Think of states as rooms and transitions as people moving between rooms:

- `π[i]` = fraction of people in room `i`
- `P[i,j]` = probability someone in room `i` moves to room `j`

**Stationarity means:** 
- The flow of people leaving each room = flow of people entering that room
- The distribution is **balanced** – no net accumulation or depletion

**Example:** 2-state weather model (Sunny ↔ Rainy)
```
If π = [0.6, 0.4]:
- 60% sunny days, 40% rainy days in the long run
- Flow from S→R = 0.6 × P[S,R]
- Flow from R→S = 0.4 × P[R,S]
- These flows balance out!
```

---

### 🔑 Existence and Uniqueness

**When does `π` exist?**
- Finite Markov chains **always** have at least one stationary distribution
- But there might be multiple!

**When is `π` unique?**
- If the chain is **irreducible** (can reach any state from any state)
- Then there is **exactly one** stationary distribution
- Moreover, if also **aperiodic**, then `μ_t → π` as `t → ∞` from any starting distribution!

**Key theorem (Ergodic Theorem):**
> If a chain is **irreducible** and **aperiodic** (called "ergodic"), then:
> 1. Unique stationary distribution `π` exists
> 2. `P^t[i,j] → π[j]` as `t → ∞` (regardless of starting state `i`)
> 3. Time averages = space averages

---

### 🛠️ Two Methods to Compute `π`

#### **Method 1: Solve Linear System (Exact)**

Solve the equation `πP = π` subject to `Σᵢ π[i] = 1`.

This is equivalent to:
- `(P^T - I)π^T = 0` (transpose form)
- Plus the constraint `Σᵢ π[i] = 1`

**Pros:**
- Exact solution (up to numerical precision)
- Works for small to medium chains (k ≤ 1000)
- Doesn't require irreducibility or aperiodicity

**Cons:**
- Computationally expensive for large k (need to solve k×k linear system)
- Can be numerically unstable for ill-conditioned matrices

---

#### **Method 2: Power Iteration (Iterative)**

Start with any distribution `μ_0` (e.g., uniform), then iterate:
$$\mu_{t+1} = \mu_t P$$

For ergodic chains, `μ_t → π` as `t → ∞`.

**Pros:**
- Simple to implement
- Works for very large chains
- Only needs matrix-vector multiplication (fast!)
- Intuitive: simulates long-run behavior

**Cons:**
- Only works for ergodic chains (irreducible + aperiodic)
- Convergence can be slow if P has eigenvalues close to 1
- Gives approximate solution (stop when change is small)

---

### 🎯 Which Method Should You Use?

| Situation | Recommended Method |
|-----------|-------------------|
| Small chain (k < 100) | Linear solve (exact and fast) |
| Large chain (k > 1000) | Power iteration |
| Don't know if ergodic | Linear solve (more robust) |
| Need exact answer | Linear solve |
| Just need approximation | Power iteration |
| Periodic chain | Linear solve (power iteration won't converge!) |

---

### 📈 Interpreting `π`

Once you have `π`, it tells you:
1. **Long-run probabilities:** Fraction of time spent in each state
2. **Limiting distribution:** Where the chain "settles down" (if ergodic)
3. **Expected rewards:** If state `i` gives reward `r_i`, average reward = `Σᵢ π[i] r_i`

In [ ]:
def stationary_distribution_solve(P: np.ndarray) -> np.ndarray:
    """Solve πP=π with sum(π)=1 (works well for small k)."""
    P = ensure_row_stochastic(P)
    k = P.shape[0]
    A = (P.T - np.eye(k))
    # Replace one equation with normalization sum π = 1
    A[-1, :] = 1.0
    b = np.zeros(k)
    b[-1] = 1.0
    pi = np.linalg.solve(A, b)
    pi = np.clip(pi, 0.0, None)
    pi = pi / pi.sum()
    return pi

def stationary_distribution_power(P: np.ndarray, tol: float = 1e-12, max_iter: int = 200_000) -> np.ndarray:
    """Power method: μ <- μP starting from uniform; good for ergodic chains."""
    P = ensure_row_stochastic(P)
    k = P.shape[0]
    mu = np.ones(k) / k
    for _ in range(max_iter):
        mu_next = mu @ P
        if np.max(np.abs(mu_next - mu)) < tol:
            mu = mu_next
            break
        mu = mu_next
    mu = np.clip(mu, 0.0, None)
    return mu / mu.sum()

pi_solve = stationary_distribution_solve(P_demo)
pi_power = stationary_distribution_power(P_demo)
pi_solve, pi_power


## 6) Reversibility: Time-Reversible Chains

### 🎯 What is Reversibility?

A distribution `π` is **reversible** (satisfies **detailed balance**) for transition matrix `P` if:
$$\pi(x) \cdot P(x,y) = \pi(y) \cdot P(y,x) \quad \text{for all states } x, y$$

**Physical interpretation:**
- `π(x) P(x,y)` = flow from state `x` to state `y`
- `π(y) P(y,x)` = flow from state `y` to state `x`
- **Detailed balance:** Each pair of states has **balanced bidirectional flow**

Think of it like traffic: the number of cars going from city A to city B equals the number going from B to A.

---

### 🔄 Why "Reversible"?

If a chain with stationary distribution `π` satisfies detailed balance, then **running the chain backwards in time looks statistically identical to running it forwards!**

More precisely: if you record a long trajectory and play it backwards, you can't tell which direction is "forward" in time.

**Examples of reversible chains:**
- Random walk on an undirected graph (symmetric movement)
- Metropolis-Hastings MCMC algorithm (by design)
- Many physical systems in thermal equilibrium

**Examples of non-reversible chains:**
- Web surfing (links are directional – more incoming than outgoing for popular pages)
- Biological evolution (mutations are often directional)
- Economic flows (money tends to concentrate)

---

### ✅ Key Theorem (Proposition 7.27)

**If `π` satisfies detailed balance for `P`, then `π` is stationary.**

**Proof sketch:**
```
(πP)[y] = Σₓ π(x) P(x,y)         (definition of matrix multiplication)
        = Σₓ π(y) P(y,x)         (by detailed balance)
        = π(y) Σₓ P(y,x)         (factor out π(y))
        = π(y) × 1               (row y of P sums to 1)
        = π(y)                   ✅
```

So detailed balance is a **sufficient condition** for stationarity (but not necessary – stationary distributions can exist without detailed balance).

---

### 🛠️ Using Reversibility

**Why check for reversibility?**
1. **Easy verification:** If you have a candidate `π`, checking detailed balance is simple
2. **Constructive approach:** Design MCMC algorithms by enforcing detailed balance
3. **Analytical insights:** Reversible chains have special algebraic properties

**How to check:**
Verify that the matrix `Π P` is symmetric, where `Π = diag(π)`.
- `(Π P)[x,y] = π(x) P(x,y)`
- `(Π P)[y,x] = π(y) P(y,x)`
- Symmetric means `(Π P)[x,y] = (Π P)[y,x]` ⟺ detailed balance

In [ ]:
def is_reversible(P: np.ndarray, pi: np.ndarray, tol: float = 1e-10) -> bool:
    P = ensure_row_stochastic(P)
    pi = np.asarray(pi, dtype=float).reshape(-1)
    pi = pi / pi.sum()
    k = P.shape[0]
    if pi.shape[0] != k:
        raise ValueError("pi length must match P dimension.")
    lhs = pi[:, None] * P
    rhs = pi[None, :] * P.T
    return bool(np.max(np.abs(lhs - rhs)) <= tol)

is_reversible(P_demo, pi_solve)


## 7) Random Walk on an Undirected Graph

### 🎯 What is a Random Walk on a Graph?

Given an **undirected connected graph** `G = (V, E)`:
- **Vertices (V):** Represent states (e.g., web pages, locations, people)
- **Edges (E):** Represent connections (e.g., hyperlinks, roads, friendships)

A **random walk** on this graph moves from vertex to vertex by:
1. Choose one of the current vertex's neighbors **uniformly at random**
2. Move to that neighbor
3. Repeat

---

### 📐 Transition Probabilities

From vertex `v_i`, the probability of moving to `v_j` is:
$$P(v_i, v_j) = \begin{cases}
\frac{1}{\deg(v_i)} & \text{if } \{v_i, v_j\} \in E \text{ (they're neighbors)} \\
0 & \text{otherwise}
\end{cases}$$

Where `deg(v_i)` = number of edges connected to `v_i` (the **degree**).

**Example:** 4-vertex graph
```
    1 --- 2
    |     |
    0 --- 3

Degrees: deg(0)=2, deg(1)=2, deg(2)=2, deg(3)=2

From vertex 0 (neighbors: 1, 3):
  P[0,1] = 1/2, P[0,3] = 1/2, P[0,2] = 0, P[0,0] = 0
```

---

### 🎲 Stationary Distribution: Proportional to Degree!

For a random walk on an undirected connected graph, the stationary distribution is:
$$\pi(v_i) = \frac{\deg(v_i)}{2|E|}$$

Where `2|E|` = sum of all degrees (each edge contributes to 2 vertices).

**Simplified form:**
$$\pi(v_i) = \frac{\deg(v_i)}{\sum_j \deg(v_j)}$$

**Intuition:**
- High-degree vertices (hubs) are visited **more often** in the long run
- This makes sense: more edges leading in = more likely to arrive there!

**Example:** Linear chain `0 --- 1 --- 2 --- 3 --- 4`
```
Degrees: [1, 2, 2, 2, 1]
Sum = 8

Stationary distribution:
π = [1/8, 2/8, 2/8, 2/8, 1/8] = [0.125, 0.25, 0.25, 0.25, 0.125]

The middle vertices are visited twice as often as the endpoints!
```

---

### ✅ Why is This Distribution Stationary?

This random walk chain is **reversible** with respect to the degree-based distribution!

Check detailed balance:
```
π(i) P(i,j) = [deg(i) / (2|E|)] × [1/deg(i)]     (if i and j are neighbors)
            = 1 / (2|E|)

π(j) P(j,i) = [deg(j) / (2|E|)] × [1/deg(j)]     (symmetric)
            = 1 / (2|E|)

π(i) P(i,j) = π(j) P(j,i) ✅  Detailed balance holds!
```

By Proposition 7.27, this proves `π` is stationary.

---

### 🌐 Applications

1. **Network analysis:** Find important nodes (high degree = high PageRank)
2. **Sampling from graphs:** Generate representative samples
3. **Community detection:** Long trajectories tend to stay within communities
4. **Load balancing:** Distribute resources proportional to connectivity

In [ ]:
def random_walk_transition_from_edges(num_nodes: int, edges: Sequence[Tuple[int, int]]) -> np.ndarray:
    """Build transition matrix for random walk on an undirected graph."""
    if num_nodes <= 0:
        raise ValueError("num_nodes must be positive.")
    adj = [[] for _ in range(num_nodes)]
    for a, b in edges:
        if not (0 <= a < num_nodes and 0 <= b < num_nodes):
            raise ValueError("Edge has node out of range.")
        if a == b:
            continue
        adj[a].append(b)
        adj[b].append(a)
    P = np.zeros((num_nodes, num_nodes), dtype=float)
    for i in range(num_nodes):
        deg = len(adj[i])
        if deg == 0:
            raise ValueError("Graph has an isolated node; random walk would get stuck.")
        for j in adj[i]:
            P[i, j] += 1.0 / deg
    return P

def stationary_for_undirected_random_walk(num_nodes: int, edges: Sequence[Tuple[int, int]]) -> np.ndarray:
    """Return π proportional to degree for connected undirected graph random walk."""
    deg = np.zeros(num_nodes, dtype=float)
    for a, b in edges:
        if a == b:
            continue
        deg[a] += 1
        deg[b] += 1
    if np.any(deg == 0):
        raise ValueError("Graph has isolated node.")
    pi = deg / deg.sum()
    return pi

# Demo graph: 5 nodes in a chain 0-1-2-3-4
edges_demo = [(0,1),(1,2),(2,3),(3,4)]
P_rw = random_walk_transition_from_edges(5, edges_demo)
pi_deg = stationary_for_undirected_random_walk(5, edges_demo)

is_reversible(P_rw, pi_deg), pi_deg


### Random Surfer / PageRank-Style Chain

### 🎯 The Problem with Basic Random Walks

Basic random walks on graphs can have issues:
1. **Dead ends (sinks):** Pages with no outgoing links → walker gets stuck!
2. **Periodic behavior:** Bipartite graphs → oscillation, no convergence
3. **Disconnected components:** Can't reach all states

**Real-world example:** Web graph
- Many pages have no outgoing links (dead ends)
- Link structure is highly irregular
- Original PageRank needs to handle these cases

---

### 💡 Solution: Random Surfer with Teleportation

The **random surfer model** blends two behaviors:
1. **With probability `α`:** Follow a link from the current page (normal random walk)
2. **With probability `1-α`:** "Teleport" to a **random page** uniformly

**Modified transition matrix:**
$$P' = \alpha P + (1-\alpha) \frac{1}{k} \mathbf{1}\mathbf{1}^T$$

Where:
- `P` = base transition matrix (from graph structure)
- `k` = number of states
- `1/k 𝟙𝟙ᵀ` = uniform distribution matrix (all entries = 1/k)
- `α` = damping factor (typically 0.85 in PageRank)

---

### 🔑 Why This Works

**Guarantees ergodicity:**
- **Irreducible:** Can reach any state from any state (via teleportation)
- **Aperiodic:** Self-loops exist (stay in same state via teleportation with prob. `(1-α)/k > 0`)
- ➜ Unique stationary distribution exists!
- ➜ Power method converges!

**Physical interpretation:**
- Models realistic web surfing: users sometimes type random URLs instead of following links
- The `(1-α)` probability prevents getting stuck in dead ends
- Higher `α` = more emphasis on link structure; lower `α` = more uniform exploration

---

### 🎛️ The Parameter `α` (Damping Factor)

**Typical value:** `α = 0.85` (from original PageRank paper)

| α value | Interpretation |
|---------|----------------|
| α = 1.0 | Pure random walk (may have problems) |
| α = 0.85 | 85% follow links, 15% random jump (standard) |
| α = 0.5 | Equal weight to structure vs. uniform |
| α = 0.0 | Completely uniform (ignores graph structure!) |

**Trade-offs:**
- **Higher α:** Stationary distribution more influenced by graph structure, but slower convergence
- **Lower α:** Faster convergence, more uniform distribution, less influenced by topology

---

### 🌐 PageRank Connection

This is essentially **Google's PageRank algorithm**!
- States = web pages
- Edges = hyperlinks
- Stationary distribution `π[i]` = "importance" or "rank" of page `i`
- Sort pages by `π[i]` to rank search results

**Why it works:**
- Important pages have many incoming links (high degree)
- Links from important pages count more (recursive definition!)
- Stationary distribution captures this recursive importance naturally

In [ ]:
def random_surfer_transition(P: np.ndarray, alpha: float = 0.85) -> np.ndarray:
    """Blend a base transition matrix with uniform teleportation."""
    if not (0.0 < alpha < 1.0):
        raise ValueError("alpha must be in (0,1).")
    P = ensure_row_stochastic(P)
    k = P.shape[0]
    U = np.ones((k, k), dtype=float) / k
    return alpha * P + (1 - alpha) * U

P_surf = random_surfer_transition(P_rw, alpha=0.85)
pi_surf = stationary_distribution_power(P_surf)
pi_surf


## 8) Exercise: Text as a Markov Chain (Word-Level)

### 🎯 The Idea: Simple Language Modeling

Can we model language as a Markov chain? The idea:
- **States** = words in vocabulary
- **Transitions** = probability of word `w_2` following word `w_1`
- **Process:** Given text, estimate `P(w_2 | w_1)` and use it to generate new sentences

This is a **very simple language model** (1st-order Markov chain = bigram model).

---

### 📚 Example

Given text: "the cat sat on the mat the cat sat on the rug"

**Word transitions:**
```
"the" → "cat" (twice)
"the" → "mat" (once)  
"the" → "rug" (once)
"cat" → "sat" (twice)
"sat" → "on" (twice)
"on" → "the" (twice)
"mat" → "the" (once)
"rug" → (end)
```

**Transition probabilities from "the":**
```
P("the" → "cat") = 2/4 = 0.5
P("the" → "mat") = 1/4 = 0.25
P("the" → "rug") = 1/4 = 0.25
```

---

### 🔧 The Algorithm

#### **Step 1: Tokenize**
Break text into words (tokens):
```
"The cat sat!" → ["the", "cat", "sat"]  (lowercase, remove punctuation)
```

#### **Step 2: Count Transitions**
Build a count matrix:
```
count[w1, w2] = number of times w2 follows w1 in the text
```

#### **Step 3: Normalize to Get Probabilities**
For each word `w1`, normalize its row:
```
P[w1, w2] = count[w1, w2] / Σ_{w} count[w1, w]
```

Now `P` is a row-stochastic transition matrix!

#### **Step 4 (Optional): Smoothing**
Problem: What if we never saw "the" → "zebra" in training? `P["the", "zebra"] = 0`, even if it's grammatically valid.

**Solution:** Add a small constant (Laplace smoothing):
```
count[w1, w2] += ε  (e.g., ε = 0.01)
```

This gives small probability to unseen transitions.

---

### 📊 What Can We Do?

1. **Compute transition probabilities:**
   ```python
   P("the" → "cat") = ?
   ```

2. **Generate sentences:**
   - Start with a word (e.g., "the")
   - Sample next word from `P["the", :]`
   - Continue for `n` words
   - Result: "the cat sat on the mat ..." (machine-generated text!)

3. **Evaluate perplexity:**
   How well does the model predict held-out text?

---

### 🎨 Try It Yourself!

**Experiment ideas:**
- Use a real book (e.g., Pride & Prejudice from Project Gutenberg)
- Try different smoothing values
- Compare generated text with/without smoothing
- Build a 2nd-order model (consider last 2 words instead of 1)

**Limitations of 1st-order models:**
- No long-range dependencies (can't remember context from 10 words ago)
- Grammar violations are common
- Modern language models (GPT, etc.) use much more sophisticated architectures!

But this is a great starting point to understand sequence modeling. 🚀

In [ ]:
def tokenize_simple(text: str, lowercase: bool = True) -> List[str]:
    """Very simple tokenizer: keeps letters and apostrophes, splits on others."""
    if lowercase:
        text = text.lower()
    tokens = []
    word = []
    for ch in text:
        if ch.isalpha() or ch == "'":
            word.append(ch)
        else:
            if word:
                tokens.append("".join(word))
                word = []
    if word:
        tokens.append("".join(word))
    return tokens

def estimate_transition_matrix_from_tokens(
    tokens: Sequence[str],
    smoothing: float = 0.0,
    min_count: int = 1,
) -> Tuple[np.ndarray, Dict[str, int], List[str]]:
    """Estimate word-level Markov chain P from tokens (Laplace smoothing optional)."""
    # Build vocabulary
    counts = {}
    for w in tokens:
        counts[w] = counts.get(w, 0) + 1
    vocab = [w for w, c in counts.items() if c >= min_count]
    vocab.sort()
    idx = {w: i for i, w in enumerate(vocab)}
    k = len(vocab)
    if k == 0:
        raise ValueError("No tokens after applying min_count.")

    C = np.full((k, k), smoothing, dtype=float)
    for a, b in zip(tokens[:-1], tokens[1:]):
        if a in idx and b in idx:
            C[idx[a], idx[b]] += 1.0
    P = normalize_rows(C)
    return P, idx, vocab

def transition_prob(P: np.ndarray, idx: Dict[str, int], a: str, b: str) -> float:
    """One-step probability P(a->b) if both in vocab."""
    if a not in idx or b not in idx:
        return 0.0
    return float(P[idx[a], idx[b]])

def generate_tokens_from_chain(
    P: np.ndarray,
    start_state: int,
    n_tokens: int,
    seed: Optional[int] = 0,
) -> List[int]:
    """Generate a sequence of state indices of length n_tokens."""
    traj = simulate_markov_chain(P, x0=start_state, n_steps=max(0, n_tokens - 1), seed=seed)
    return traj.tolist()

def generate_sentence(
    P: np.ndarray,
    vocab: List[str],
    start_word: str,
    n_words: int = 15,
    seed: Optional[int] = 0,
) -> str:
    """Generate a sentence-like string of n_words, starting from start_word (must exist in vocab)."""
    idx = {w:i for i,w in enumerate(vocab)}
    if start_word not in idx:
        raise ValueError(f"start_word '{start_word}' not in vocabulary.")
    states = generate_tokens_from_chain(P, idx[start_word], n_words, seed=seed)
    return " ".join(vocab[s] for s in states)

# Demo with a tiny text sample (replace with file contents)
demo_text = "Lady Catherine was not pleased. Lady Catherine spoke to her niece."
tokens = tokenize_simple(demo_text, lowercase=True)
P_txt, idx_txt, vocab_txt = estimate_transition_matrix_from_tokens(tokens, smoothing=0.1)

print("P(the->her) =", transition_prob(P_txt, idx_txt, "the", "her"))
print(generate_sentence(P_txt, vocab_txt, start_word="lady", n_words=10, seed=1))


## 9) Exercise: Branching Process (Galton-Watson)

### 🎯 What is a Branching Process?

A **branching process** models **population dynamics** where individuals reproduce independently:
- Start with `Z_0` individuals (e.g., `Z_0 = 1`)
- Each individual produces a **random number of offspring** (according to some distribution)
- All individuals reproduce simultaneously, then die
- The next generation has `Z_1` individuals (sum of all offspring)
- Repeat: `Z_0 → Z_1 → Z_2 → ...`

**Key question:** Does the population die out (go **extinct**) or grow forever?

---

### 👨‍👩‍👧‍👦 Example: Simple Reproduction Model

Each individual has 0, 1, or 2 offspring with probabilities:
```
P(0 offspring) = p_0
P(1 offspring) = p_1  
P(2 offspring) = p_2
```

**Generation dynamics:**
```
Generation 0: Z_0 = 1 individual

Generation 1: The individual has X offspring
              Z_1 = X  (X ∈ {0, 1, 2})

Generation 2: Each of the Z_1 individuals has offspring independently
              Z_2 = X_1 + X_2 + ... + X_{Z_1}
```

---

### 🧬 Why is This a Markov Chain?

The next population size `Z_{t+1}` depends **only on the current size** `Z_t`, not on history!
- **State space:** {0, 1, 2, 3, ...} (non-negative integers)
- **State 0 is absorbing:** Once extinct (Z_t = 0), stays extinct forever
- **Markov property holds:** P(Z_{t+1} | Z_t, Z_{t-1}, ..., Z_0) = P(Z_{t+1} | Z_t)

---

### 📊 Key Quantity: Mean Offspring `μ`

Let `μ = E[X]` be the expected number of offspring per individual:
$$\mu = 0 \cdot p_0 + 1 \cdot p_1 + 2 \cdot p_2$$

**Three regimes:**

1. **Subcritical (`μ < 1`):** 
   - Population **almost surely goes extinct**
   - E.g., each individual has < 1 child on average → shrinking population

2. **Critical (`μ = 1`):**
   - Population **almost surely goes extinct** (but takes longer)
   - E.g., exactly replacement level, but randomness causes eventual extinction

3. **Supercritical (`μ > 1`):**
   - **Positive probability of survival** (but still might go extinct!)
   - Two outcomes: either goes extinct OR grows exponentially
   - E.g., each individual has > 1 child on average → growth possible

---

### 🎲 Extinction Probability

Let `q` = probability of eventual extinction starting from 1 individual.

**Theoretical result:** `q` is the **smallest non-negative solution** to:
$$q = \sum_{k=0}^{\infty} p_k q^k$$

For our 3-state model (`k ∈ {0,1,2}`):
$$q = p_0 + p_1 q + p_2 q^2$$

**Special cases:**
- If `μ ≤ 1`: Then `q = 1` (extinction is certain)
- If `μ > 1`: Then `q < 1` (survival possible!)

---

### ⏱️ Extinction Time (Lifetime)

Even if extinction is certain, **how long does it take?**

Define `τ` = first time when `Z_t = 0` (extinction time or "lifetime").

**Questions we can answer via simulation:**
1. What is `E[τ]` (expected lifetime)?
2. What is the distribution of `τ`?
3. How does `E[τ]` depend on `p_0, p_1, p_2`?

---

### 🧪 Simulation Approach

1. **Start:** `Z_0 = 1`
2. **At each generation:**
   - If `Z_t = 0`: Stop (extinct!)
   - Otherwise: For each individual, draw offspring count from {0,1,2} with probs. `(p_0, p_1, p_2)`
   - Sum up: `Z_{t+1} = ` total offspring
3. **Record:** Trajectory `[Z_0, Z_1, Z_2, ..., Z_τ]` where `Z_τ = 0`

Run many simulations to estimate:
- `E[τ]` ≈ average extinction time over simulations
- `P(τ > t)` ≈ fraction of simulations still alive at time `t`

---

### 🌍 Real-World Applications

- **Biology:** Species extinction, epidemic spread (each infected person infects others)
- **Nuclear physics:** Neutron chain reactions
- **Genealogy:** Family name survival (surnames passed through male lineage)
- **Startups:** Company survival (each company spawns spin-offs or fails)

This is a fundamental model in probability theory with deep connections to random trees, percolation, and critical phenomena!

In [ ]:
def simulate_branching_process(
    p: Sequence[float],
    Z0: int = 1,
    max_steps: int = 10_000,
    seed: Optional[int] = 0,
) -> np.ndarray:
    """Simulate branching process until extinction or max_steps."""
    p = np.asarray(p, dtype=float)
    if p.shape[0] != 3:
        raise ValueError("p must have length 3 for offspring {0,1,2}.")
    if np.any(p < 0) or not np.isclose(p.sum(), 1.0):
        raise ValueError("p must be a probability vector summing to 1.")
    if Z0 < 0:
        raise ValueError("Z0 must be >= 0.")
    rng = np.random.default_rng(seed)
    Z = [int(Z0)]
    for _ in range(max_steps):
        if Z[-1] == 0:
            break
        # offspring for each of Z[-1] individuals
        offspring = rng.choice([0,1,2], size=Z[-1], p=p)
        Z.append(int(np.sum(offspring)))
    return np.array(Z, dtype=int)

def extinction_time(Z_path: np.ndarray) -> Optional[int]:
    """Return first n with Z_n=0; None if not extinct in simulated path."""
    zeros = np.where(Z_path == 0)[0]
    return int(zeros[0]) if zeros.size else None

def estimate_expected_lifetime(
    p: Sequence[float],
    n_sims: int = 5000,
    Z0: int = 1,
    max_steps: int = 10_000,
    seed: Optional[int] = 0,
) -> float:
    """Estimate E[tau] where tau is extinction time."""
    rng = np.random.default_rng(seed)
    taus = []
    for s in range(n_sims):
        Z = simulate_branching_process(p, Z0=Z0, max_steps=max_steps, seed=int(rng.integers(0, 2**31-1)))
        tau = extinction_time(Z)
        if tau is None:
            tau = max_steps  # censored; you can change this rule if you want
        taus.append(tau)
    return float(np.mean(taus))

# Demo with p=(1/3,1/3,1/3)
p = (1/3, 1/3, 1/3)
Z = simulate_branching_process(p, max_steps=200, seed=3)
tau = extinction_time(Z)
tau, Z[:20]


In [ ]:
# Plot one sample path
plt.figure()
plt.plot(Z, marker='o', linewidth=1)
plt.title("Branching process sample path")
plt.xlabel("n")
plt.ylabel("Z_n")
plt.show()

print("Estimated expected lifetime (rough):", estimate_expected_lifetime(p, n_sims=2000, max_steps=500, seed=1))


---

## 📚 Summary: Key Concepts and Their Relationships

### 🎯 The Big Picture

**Markov chains** model systems that transition between states with **memoryless** transitions (future depends only on present, not past).

---

### 🗺️ Concept Map

```
                    MARKOV CHAIN
                         |
        ┌────────────────┼────────────────┐
        |                |                |
   TRANSITION       DISTRIBUTIONS    SIMULATION
    MATRIX P          μ_t = μ_0 P^t      (RMR)
        |                                  |
   Row-stochastic                   Inversion
   (rows sum to 1)                   Sampling
        |
        |
    STRUCTURE                      LONG-RUN BEHAVIOR
        |                                  |
   ┌────┴────┐                    ┌───────┴───────┐
   |         |                    |               |
Irreducible Period          Stationary π     Reversibility
   |         |              (πP = π)          (detailed balance)
   |         |                    |
   └─────────┴────────────────────┘
              |
         ERGODIC CHAIN
    (irreducible + aperiodic)
              |
    ┌─────────┴─────────┐
    |                   |
Unique π          μ_t → π
Convergence      Power method
```

---

### 🔑 Essential Properties

| Property | Definition | Why It Matters |
|----------|-----------|----------------|
| **Markov property** | P(X_{t+1} \| X_t, ..., X_0) = P(X_{t+1} \| X_t) | Memoryless: only current state matters |
| **Row-stochastic** | Each row of P sums to 1 | Valid probability distribution for transitions |
| **Irreducible** | Can reach any state from any state | ➜ Unique stationary distribution |
| **Aperiodic** | Period = 1 (no cyclic structure) | ➜ Smooth convergence to π |
| **Ergodic** | Irreducible + aperiodic | ➜ μ_t → π guaranteed! |
| **Stationary π** | πP = π | Equilibrium distribution (doesn't change) |
| **Reversible** | π(i)P(i,j) = π(j)P(j,i) | Detailed balance ➜ easy to verify π |

---

### 🎓 Key Theorems to Remember

1. **Chapman-Kolmogorov:** `μ_t = μ_0 P^t` (distributions evolve by matrix multiplication)

2. **Ergodic Theorem:** If chain is irreducible + aperiodic, then:
   - Unique stationary distribution π exists
   - μ_t → π as t → ∞ (regardless of starting point)
   - Time averages = space averages

3. **Reversibility ⇒ Stationarity:** If π satisfies detailed balance, then πP = π

4. **Random Walk on Graph:** Stationary distribution is proportional to degree

---

### 🛠️ Practical Workflow

When analyzing a Markov chain:

1. **Validate:** Check that P is row-stochastic
2. **Structure:** Check if irreducible (all states communicate)
3. **Period:** Check if aperiodic (returns not cyclic)
4. **Stationary distribution:**
   - If ergodic: Use power method (fast, simple)
   - Otherwise: Solve linear system (exact)
5. **Reversibility:** Check detailed balance if you have a candidate π
6. **Simulate:** Use RMR for trajectories and empirical estimates

---

### 📈 When to Use Which Method?

#### **Computing Stationary Distribution:**
- Small chain (k < 100): **Linear solve** (exact)
- Large chain: **Power iteration** (if ergodic)
- Periodic chain: **Linear solve** (power iteration won't converge!)

#### **Analyzing Structure:**
- Small chain: Check **all pairs** of states
- Large chain: Use **graph algorithms** (DFS, strongly connected components)

#### **Simulation:**
- Few steps: **Direct matrix multiplication** (μ_t = μ_0 P^t)
- Many steps or trajectories: **RMR simulation** (faster, more flexible)

---

### 🚀 Next Steps & Extensions

#### **Advanced Topics to Explore:**

1. **Continuous-time Markov chains:**
   - Transitions happen at random times (not discrete steps)
   - Uses rate matrices instead of transition matrices

2. **Markov Chain Monte Carlo (MCMC):**
   - Design chains to sample from target distributions
   - Metropolis-Hastings, Gibbs sampling
   - Applications: Bayesian inference, computational physics

3. **Hidden Markov Models (HMMs):**
   - States are hidden, only observations are visible
   - Applications: speech recognition, bioinformatics

4. **Markov Decision Processes (MDPs):**
   - Add actions and rewards to Markov chains
   - Basis of reinforcement learning (Q-learning, policy gradient)

5. **Infinite state spaces:**
   - Countably infinite (birth-death processes, queues)
   - Uncountable (continuous state spaces, diffusions)

6. **Higher-order models:**
   - 2nd-order: P(X_t | X_{t-1}, X_{t-2})
   - Variable-order Markov models
   - Neural language models

---

### 📖 Recommended Resources

**Books:**
- *Markov Chains* by J.R. Norris (rigorous mathematical treatment)
- *Introduction to Probability Models* by Sheldon Ross (applications-focused)
- *Monte Carlo Statistical Methods* by Robert & Casella (MCMC focus)

**Online:**
- 3Blue1Brown: "Markov Chains" (YouTube - great visual intuition)
- Brilliant.org: Interactive Markov chain problems
- setosa.io: Interactive visualizations of random walks

**Practice:**
- Implement MCMC samplers (Metropolis-Hastings)
- Analyze real-world networks (social, biological, web)
- Build higher-order language models
- Study queueing theory and applications

---

## 🎉 Congratulations!

You now have a complete toolkit for working with finite Markov chains. These concepts form the foundation for:
- Probabilistic modeling
- Time series analysis  
- Machine learning (HMMs, RL)
- Network science
- Computational statistics (MCMC)

**Keep exploring, keep experimenting, and have fun with Markov chains!** 🚀